#12.1 Your First Agent

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def multiply(a, b):
    return a * b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=multiply,
            name="Multiplication Calculator",
            description="Use this to perform multiplications of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
)

prompt = "Cost of $2.4 x $45.3 x $23.7"
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)
# agent.invoke(prompt)

AIMessage(content='To find the cost of $2.4 multiplied by $45.3 multiplied by $23.7, you would simply multiply the three numbers together:\n\n$2.4 x $45.3 x $23.7 = $2.4 x 45.3 x 23.7 = $258.48 x 23.7 = $6130.776\n\nTherefore, the cost of $2.4 multiplied by $45.3 multiplied by $23.7 is $6130.776.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 23, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c851ab21-b463-4b5c-83d5-d7a2b7451c16-0')

In [3]:
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought:
To find the cost of $2.4 multiplied by $45.3 and then multiplied by $23.7, I can use the Multiplication Calculator tool.

Action:
```
{
  "action": "Multiplication Calculator",
  "action_input": {
    "a": 2.4,
    "b": 45.3
  }
}
```
Observation: 108.71999999999998
Thought:Now that we have the result of the first multiplication, we can proceed to multiply it by $23.7 to find the final cost.
Action: 
```
{
  "action": "Multiplication Calculator",
  "action_input": {
    "a": 108.72,
    "b": 23.7
  }
}
```
Observation: 2576.6639999999998
Thought:Now that we have the final result of multiplying $2.4 by $45.3 and then by $23.7, we can provide the total cost.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The cost is $2576.66"
}
```

> Finished chain.


{'input': 'Cost of $2.4 x $45.3 x $23.7', 'output': 'The cost is $2576.66'}